adapted from https://github.com/kelvins/steganography/blob/main/steganography.py

In [18]:
import click
from PIL import Image
import numpy as np
import os
from sklearn.metrics import mean_squared_error as mse

## LSB implementation

In [19]:
class Steganography:

    @staticmethod
    def __int_to_bin(rgb):
        """Convert an integer tuple to a binary (string) tuple.
        :param rgb: An integer tuple (e.g. (220, 110, 96))
        :return: A string tuple (e.g. ("00101010", "11101011", "00010110"))
        """
        r, g, b = rgb
        return (f'{r:08b}',
                f'{g:08b}',
                f'{b:08b}')

    @staticmethod
    def __bin_to_int(rgb):
        """Convert a binary (string) tuple to an integer tuple.
        :param rgb: A string tuple (e.g. ("00101010", "11101011", "00010110"))
        :return: Return an int tuple (e.g. (220, 110, 96))
        """
        r, g, b = rgb
        return (int(r, 2),
                int(g, 2),
                int(b, 2))

    @staticmethod
    def __merge_rgb(rgb1, rgb2):
        """Merge two RGB tuples.
        :param rgb1: A string tuple (e.g. ("00101010", "11101011", "00010110"))
        :param rgb2: Another string tuple
        (e.g. ("00101010", "11101011", "00010110"))
        :return: An integer tuple with the two RGB values merged.
        """
        r1, g1, b1 = rgb1
        r2, g2, b2 = rgb2
        rgb = (r1[:4] + r2[:4],
               g1[:4] + g2[:4],
               b1[:4] + b2[:4])
        return rgb

    @staticmethod
    def merge(img1, img2):
        """Merge two images. The second one will be merged into the first one.
        :param img1: First image
        :param img2: Second image
        :return: A new merged image.
        """

        # Check the images dimensions
        if img2.size[0] > img1.size[0] or img2.size[1] > img1.size[1]:
            raise ValueError('Image 2 should not be larger than Image 1!')

        # Get the pixel map of the two images
        pixel_map1 = img1.load()
        pixel_map2 = img2.load()

        # Create a new image that will be outputted
        new_image = Image.new(img1.mode, img1.size)
        pixels_new = new_image.load()

        for i in range(img1.size[0]):
            for j in range(img1.size[1]):
                rgb1 = Steganography.__int_to_bin(pixel_map1[i, j])

                # Use a black pixel as default
                rgb2 = Steganography.__int_to_bin((0, 0, 0))

                # Check if the pixel map position is valid for the second image
                if i < img2.size[0] and j < img2.size[1]:
                    rgb2 = Steganography.__int_to_bin(pixel_map2[i, j])

                # Merge the two pixels and convert it to a integer tuple
                rgb = Steganography.__merge_rgb(rgb1, rgb2)

                pixels_new[i, j] = Steganography.__bin_to_int(rgb)

        return new_image

    @staticmethod
    def unmerge(img):
        """Unmerge an image.
        :param img: The input image.
        :return: The unmerged/extracted image.
        """

        # Load the pixel map
        pixel_map = img.load()

        # Create the new image and load the pixel map
        new_image = Image.new(img.mode, img.size)
        pixels_new = new_image.load()

        # Tuple used to store the image original size
        original_size = img.size

        for i in range(img.size[0]):
            for j in range(img.size[1]):
                # Get the RGB (as a string tuple) from the current pixel
                r, g, b = Steganography.__int_to_bin(pixel_map[i, j])

                # Extract the last 4 bits (corresponding to the hidden image)
                # Concatenate 4 zero bits because we are working with 8 bit
                rgb = (r[4:] + '0000',
                       g[4:] + '0000',
                       b[4:] + '0000')

                # Convert it to an integer tuple
                pixels_new[i, j] = Steganography.__bin_to_int(rgb)

                # If this is a 'valid' position, store it
                # as the last valid position
                if pixels_new[i, j] != (0, 0, 0):
                    original_size = (i + 1, j + 1)

        # Crop the image based on the 'valid' pixels
        new_image = new_image.crop((0, 0, original_size[0], original_size[1]))

        return new_image

def cli():
    pass

def merge(img1, img2, output):
    merged_image = Steganography.merge(Image.open(img1), Image.open(img2))
    merged_image.save(output)

def unmerge(img, output):
    unmerged_image = Steganography.unmerge(Image.open(img))
    unmerged_image.save(output)

In [30]:
############################################# This was used to resize the images at some point, but I have them saved
############################################# now, so just ignore this error please... was just trying to edit below
#kodak = os.listdir("kodakSet/temp/")
#for i in range(len(kodak)):
 #   temp = Image.open("kodakSet/temp/"+str(kodak[i]))
  #  #to resize the images
   # new_img = temp.crop((50,50,500,500)).resize((256,256))
    ##slightly messed up on the names for saving the images, but can still make do with what I have.
    #new_img.save("kodakSet/resized" + str(i) + ".png")

PermissionError: [Errno 13] Permission denied: 'kodakSet/temp/images'

In [ ]:
#test to make sure I know what to do
#img1 = Image.open("kodakSet/resized/0.png")
#img2 = Image.open("kodakSet/resized/1.png")
#output = "kodakSet/temp/merge1.png"
#output2 = "kodakSet/temp/unmerge1.png"
#merge("kodakSet/resized/0.png","kodakSet/resized/1.png","kodakSet/temp/merge1.png")
#unmerge("kodakSet/temp/merge1.png", "kodakSet/temp/unmerge1.png")

In [28]:
loss = []
for i in range(20):
    #just doing 20 merge and unmerge. Technically possible for repeats to occur, hopefully not many
    x = np.random.randint(0,24,2)
    merge("kodakSet/resized/" + str(x[0]) + ".png", "kodakSet/resized/" + str(x[1]) + ".png", "kodakSet/merge/" + str(i) + ".png")
    unmerge("kodakSet/merge/" + str(i) + ".png", "kodakSet/unmerge/" + str(i) + ".png")
    a = Image.open("kodakSet/resized/" + str(x[0]) + ".png")
    b = Image.open("kodakSet/resized/" + str(x[1]) + ".png")
    c = Image.open("kodakSet/merge/" + str(i) + ".png")
    d = Image.open("kodakSet/unmerge/" + str(i) + ".png")
    a = np.array(a).reshape(256,-1)
    b = np.array(b).reshape(256,-1)
    c = np.array(c).reshape(256,-1)
    d = np.array(d).reshape(256,-1)
    loss.append(mse(a,b)+mse(c,d))
    print("RMSE = ", mse(a,b)+mse(c,d))

RMSE =  215.82945251464844
RMSE =  216.67703247070312
RMSE =  220.96944173177084
RMSE =  199.14102172851562
RMSE =  216.25413513183594
RMSE =  203.628662109375
RMSE =  218.56898498535156
RMSE =  208.7431386311849
RMSE =  215.3519337972005
RMSE =  226.02313232421875
RMSE =  222.98048400878906
RMSE =  220.16421000162762
RMSE =  217.7460225423177
RMSE =  206.30533854166666
RMSE =  209.32330322265625
RMSE =  223.6094207763672
RMSE =  221.83853149414062
RMSE =  210.57570393880206
RMSE =  208.39427693684894
RMSE =  92.26107788085938
